**k-Means example with Iris data**

In [2]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

n = len(X)
d = len(X[0])

**k-Means by sklearn**

It's not quite stable, that's why sklearn provide args for smarter init and retry

In [8]:
from sklearn.cluster import KMeans
import numpy as np
from itertools import permutations
import time

start = time.time()
kmeans = KMeans(n_clusters=3, init='random', n_init=1).fit(X)
y_hat = kmeans.predict(X)

def eval(y_hat):
    y_hat_mapped = []

    for mapping in permutations([0,1,2]):
        map_y = lambda x: mapping[x]
        y_hat_mapped.append(np.vectorize(map_y)(y_hat))

    accuracies = [np.mean(y_pred == y) for y_pred in y_hat_mapped]
    print(f'accuracy: {max(accuracies)}')

eval(y_hat)
print(f'train-eval elapsed {time.time()-start}')

accuracy: 0.8866666666666667
train-eval elapsed 0.006981611251831055


**k-Means from scratch**

In [9]:

i_centroids = np.random.randint(low=0, high=n, size=3)
max_iter = 300

start = time.time()
centroids = X[i_centroids]
def predict(X, centroids):
    derive_dist = lambda x_row: [np.linalg.norm(x_row - centroid) for centroid in centroids]
    all_distances = np.apply_along_axis(derive_dist, arr=X, axis=1)
    return np.argmin(all_distances, axis=1)

for i in range(1, max_iter+1):
    # predict and assign
    prediction = predict(X, centroids)
    assignment =  [[] for _ in range(3)]
    for row_num, pred_row in enumerate(prediction):
        assignment[pred_row].append(row_num)

    # adjust
    new_centroids = np.array([np.mean(X[indicies], axis=0) for indicies in assignment])
    if np.array_equal(new_centroids, centroids):
        print(f'Early stop at iteration {i}')
        break

    centroids = new_centroids


y_hat = predict(X, centroids)

eval(y_hat)
print(f'train-eval elapsed {time.time()-start}')

Early stop at iteration 9
accuracy: 0.8866666666666667
train-eval elapsed 0.022937297821044922


Full Vectorization

In [24]:
import time
i_centroids = np.random.randint(low=0, high=n, size=3)
max_iter = 300

start = time.time()
centroids = X[i_centroids]
n, d = X.shape
k = 3
def predict(X, centroids):
    sq_dist = np.sum((X.reshape(n, 1, d) - centroids.reshape(k, d)) ** 2, axis=2)
    return np.argmin(sq_dist, axis=1)

for i in range(1, max_iter+1):
    # predict and assign
    prediction = predict(X, centroids) # shape n, k
    # adjust
    new_centroids = np.array([np.mean(X[prediction==clz], axis=0) for clz in range(k)])
    if np.array_equal(new_centroids, centroids):
        print(f'Early stop at iteration {i}')
        break

    centroids = new_centroids


y_hat = predict(X, centroids)
eval(y_hat)
print(f'train-eval elapsed {time.time()-start}')


Early stop at iteration 6
accuracy: 0.8866666666666667
train-eval elapsed 0.0019965171813964844
